# Ensemble Classifiers

### Introduction to Our Problem
There are literally tens of thousands of movies out there today. While some do great at the box office and bring in a lot of money, others flop making only a fraction compared to the top hits. What if we had a scientific way of accurately predicting how much revenue a movie would generate over its lifetime? Well, through machine learning we believe that we actually can!

The dataset we are using is found on <a href="https://www.kaggle.com/deepmatrix/imdb-5000-movie-dataset">Kaggle</a>. It consists of 5000+ movies scraped from the review site IMDB. There is quite a bit of data recorded for each movie and so we had a lot to work with to try to predict the next big hit. The data was collected from web scraping IMDB using a python library called "scrappy" to collect all of the data below. The features recorded for each movie are: 

Basic Info:
- movie title
- color (black and white or color)	
- duration of the movie
- director name
- gross (total revenue)
- genres (a lits of different genres ascribed to the movie)
- number of faces in movie poster
- language of the movie
- country the movie was produced in
- content rating (G, PG, PG-13, R, NC-17)
- budget
- year of release
- aspect ratio
- name of the 3rd actor
- name of the 2nd actor
- name of the 1st actor

Facebook Info:
- number of director facebook likes
- number of facebook likes for the whole cast
- number of the movie's facebook likes
- number of the 3rd actor's facebook likes
- number of the 2nd actor's facebook likes
- number of the 1st actor's facebook likes

IMDB Specific Info:
- number of imdb users who rated the movie
- number of critical reviews for the movie
- number of users who left a review
- imdb score
- top plot keywords


With all of this data collected on so many movies, we hope to be able to use this to build out a multi-layer perceptron  to accurately predict the financial success (measured in categories of gross revenue: low, low-mid, high-mid, and high) of a movie. We think that this could be a useful tool to anyone in the movie industry who is concerned with making a profit on their movie. It could also help a producer understand which of these features are the most important to an accurate prediction, what content rating is most important, how budget affects outcome, etc.


We believe that the algorithm would have to predict with a relatively low cost (under ~30) to be found useful by movie directors, producers, etc. 

### Data Pre-processing:
We made a number of changes to both the original csv obtained from kaggle before we loaded it and to the data once it was loaded in.


Pre-processing of the CSV:
- We first removed the imdb link from the csv because we knew we would never need to use that (**Note: this was the only feature removed from the csv**)
- We then went through and deleted all of the movies that were made in another country (foriegn films) we did this because we wanted to just look at American films, also because the currency units for those countries (for budget and gross) were in native currency units, not USD, and with changing exchange rates, it's not very easy to compare across countries.
- We then went through and converted all 0 values for gross, movie_facebook_likes, and director_facebook_likes to a blank value in the csv (so that it is read in as NaN by pandas), this is so that we cna more easily impute values later. Note: according to the description on the kaggle entry, because of the way the data was scraped, some movies had missing data. The Python scraper just made these values into a 0 instead of NaN.
- We then removed all movies with an undefined gross. Being the feature we are trying to predict, we should not be imputing values for gross to train our model. That will basically reduce our model to an imputation algorithm...
- We then removed all movies that were made before 1935. We did this because there were only a handful of movies ranging from 1915 to 1935, the way we are classifying budget (described below) would not work with a small sample of movies from that time period. We could have cut this number at a different year (say 1960), but we didn't want to exclude such classics as "Bambi" or "Gone With the Wind"
- Lastly, we had to adjust the gross revenue and budget values for inflation, since the movies spanned many years. For adjusting for inflation we obtained a csv of consumer price index (CPI) for every month since 1947. To simplify, we just took the value for January of that year to use for the whole year. We then took the CPI and calculated the ratio per year compared to 2017 dollars. We then took the budget and gross and multiplied them out with their appropriate ratio value. We then exported this to the csv that we use for the rest of this lab. **NB:** This was done outside of this notebook because this whole process took a very long time when it was included in the notebook when done every time.

Pre-processing of the Data:
- After the above steps, we made more edits to the data using pandas. First, we removed features that we thought would be un-useful to our prediction algorithm. We removed all features concerning facebook likes. We did this because a significant portion of the movies in the training set debuted before facebook was invented and widely adopted. While some of these movies have received retroactive "likes" on facebook, only the most famous classics received a substantial amount of retraoctive "likes". Most lesser known films received very low amounts of "likes" (presumably because modern movie watchers don't really care to search for lesser known movies on facebook, or because the movie doesn't have a facebook). For this reason we decided to remove movie_facebook_likes
- Likewise, we removed the other "likes" for the same reasons as above. For example, the esteemed director George Lucas has a total of 0 "likes" between all of his films. This feature obviously would not help us predict the profitability of movies.
- We also removed irrelevant information such as aspect_ratio, language, and country. Because we deleted all foreign films the country will always be USA. A simple filter of the data reveals that there are no more than 20 movies made in the US that use a language other than English, therefore there is not enough data to use language as training feature. However, we did not delete the movies in a different language, because most of them were famous films such as *Letters from Iwo Jima* and *The Kite Runner*. We still count them as a valuable part of the dataset, just don't find the language of particular value. Lastly, we removed aspect_ratio because that seems to be unimportant for predicting the success of a movie.
- Lastly, we removed other features that would be difficult to use in our machine learning model such as actor names and plot keywords. We initially tried to include these in our model using one-hot encoding, but the resultant array was so enormous that the model would take a very, very long time to train.

In [1]:
# Take the dataframe and adjust for inflation and then use the df_to_csv function to export to csv 
# and then export to csv and then delete code

import pandas as pd
import numpy as np
df = pd.read_csv("inflation_corrected_dataset.csv")
for x in ['movie_facebook_likes', 'director_facebook_likes', 'actor_2_facebook_likes', 
          'actor_1_facebook_likes','actor_3_facebook_likes', 'cast_total_facebook_likes',
          'aspect_ratio', 'language', 'country', 'plot_keywords', 'actor_3_name', 'actor_2_name', 'movie_title', 'genres', 'color']:
    if x in df:
        del df[x]
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3222 entries, 0 to 3221
Data columns (total 12 columns):
director_name             3222 non-null object
num_critic_for_reviews    3219 non-null float64
duration                  3221 non-null float64
gross                     3222 non-null int64
actor_1_name              3220 non-null object
num_voted_users           3222 non-null int64
facenumber_in_poster      3216 non-null float64
num_user_for_reviews      3221 non-null float64
content_rating            3196 non-null object
budget                    3062 non-null float64
title_year                3222 non-null int64
imdb_score                3222 non-null float64
dtypes: float64(6), int64(3), object(3)
memory usage: 302.1+ KB
None


Below we group the columns by director_name and then impute as many values as we can, dropping the rows where we can't impute.

In [2]:
# Tamper with the groupings to improve imputations? How do we improve how many values get imputed?
df_grouped = df.groupby(by=['director_name'])
# director_name adds about 50 rows (imputes about 50 rows and then deletes about 100)

In [3]:
df_imputed = df_grouped.transform(lambda grp: grp.fillna(grp.median()))
col_deleted = list( set(df.columns) - set(df_imputed.columns)) #in case the median op deleted columns
df_imputed[col_deleted] = df[col_deleted]

# drop rows that still have missing values after imputation
df_imputed.dropna(inplace=True)
print(df_imputed.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3127 entries, 0 to 3220
Data columns (total 12 columns):
num_critic_for_reviews    3127 non-null float64
duration                  3127 non-null float64
gross                     3127 non-null int64
num_voted_users           3127 non-null int64
facenumber_in_poster      3127 non-null float64
num_user_for_reviews      3127 non-null float64
budget                    3127 non-null float64
title_year                3127 non-null int64
imdb_score                3127 non-null float64
content_rating            3127 non-null object
director_name             3127 non-null object
actor_1_name              3127 non-null object
dtypes: float64(6), int64(3), object(3)
memory usage: 317.6+ KB
None


### Scaling the Data
Below we scale the data using the methods shown so as to not adversely affect the gamma value. We scaled down all of the numerical values to be within -1 and 1. We also one-hot encode the content rating. We forego encoding the director name or actor names because they proved to make our matrix way too large to run computations on.

In [4]:
%%time
#scaling budgets!
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_rows', 200)

budget = df_imputed['budget'].values.reshape(-1, 1)
df_imputed.reset_index(drop=True, inplace=True)
print("df: ",df_imputed.shape)

append_list = [df_imputed]

df = pd.concat(append_list, axis=1)

#one-hot encode
hot_content = pd.get_dummies(df_imputed.content_rating, prefix='contentRating')

print(df.shape)

df:  (3127, 12)
(3127, 12)
Wall time: 3.79 s


### Cutting the gross into categories
Below we cut the adjusted, scaled, gross into 4 main categories: low, low-mid, high-mid, and high. We did this because otherwise the model would not be able to produce raw gross accurately. We also used the "qcut" function to evenly distribute the classes among the classifications, because when we did a normal cut method most of the classes would fall in the lowest category and throw off our predictions.

In [5]:
from sklearn.preprocessing import LabelEncoder

spacing = np.linspace(0, max(df['gross']), 100)
labels = []

labels = ["low", "low-mid", "high-mid", "high"]
df['gross_group'] = pd.qcut(df['gross'], 4, labels=labels)


rating_group = df['gross_group'].values
rating_encoder = LabelEncoder()
rating_df = pd.DataFrame(rating_encoder.fit_transform(rating_group), columns=['encoded_gross']).astype(str)
df = pd.concat([df, rating_df], axis=1)


## Evaluation
### Choosing Evaluation Metrics

For our dataset, accuracy is not the best evaluation metric, because that does not account properly for false positives. False positives for our business case are MUCH worse than a false negative. It would be very bad to predict that a movie will gross high, when in fact it grosses lowly. However, if we predict the movie will gross low, and it ends up grossing highly, that isn't as bad, because the director will either be pleasantly surprised, or he will choose to not undertake the filming in the first place. It is better to not film and miss out on the potential money, than to undertake the film thinking that it would be lucrative, when in fact it is not.

Because we are using a multi-class classification model we can not simply use precision, recall, or f1 score, but must construct a cost matrix with different weights that correspond to the different combination of predictions and results. Below we have our cost matrix defined. As you can see we weight a false positive with a 20 and a false negative with a 6. We give them this much of a cost difference because of the aforementioned reasons about false positives. Any True predictions are a negative one, and the other numbers in the matrix are scaled appropriately dependent upon how bad they would be as a result.

In [6]:
from sklearn.metrics import confusion_matrix, roc_curve, auc

cost_matrix = np.array([-1,10, 14,20,2,-1,10,14,4,2,-1,10,6,4,2,-1]) #give a reason for why these numbers chosen
cost_matrix = cost_matrix.reshape(4,4)
print(cost_matrix)

[[-1 10 14 20]
 [ 2 -1 10 14]
 [ 4  2 -1 10]
 [ 6  4  2 -1]]


### Dividing up training/testing data

For our dataset we want to use Stratified 10-fold cross validation because it is the best and works for our dataset well. Below we create the StratifiedKFold object and then use it many times later on in the lab. We selected this method instead of a simple 80/20 split because we new that we wanted to test on multiple randomized sets of data, instead of just the same one, so as to avoid data snooping and improper parameter tuning. We chose not to use the shuffle option becuase we wanted to compare our custom implementation and scikit-learn using the same indices for training and testing data. Below we also drop a few more fields that we no longer need due to categorizing the gross, etc.

In [7]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
cv = StratifiedKFold(n_splits=10)

drop_list = ['encoded_gross', 'gross_group', 
              'gross', 'director_name', 'actor_1_name', 'content_rating']
X = df.drop(drop_list, axis=1).values
y = df['encoded_gross'].values.astype(np.float) # x and y are now np.matrices 

X = StandardScaler().fit_transform(X)

## Modeling
### Creating an Ensemble Classifier
Before we create our ensemble classifier, we run a grid search on the individual MLP classifiers to determine which parameters would be best for each classifier while still executing in a short amount of time. Since we are using MLP classifiers to create an ensemble classifier, we want to make sure that these classifiers run pretty quickly.

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

test_params = {
    "hidden_layer_sizes": [10, 20, 30, 40, 50],
    "activation": ['logistic', 'tanh', 'relu'],
    "solver": ['lbfgs', 'adam']
}
params = {
    "alpha": 1e-4,
    "batch_size": "auto",
    "learning_rate": "constant",
    "learning_rate_init": 0.1,
    "power_t": 0.0,
    "max_iter": 75,
    "shuffle": True,
    "tol":0,
    "verbose": False,
    "warm_start": False,
    "momentum": 0.9,
    "nesterovs_momentum": False,
    "early_stopping": False,
    "validation_fraction": 0.0,
    "beta_1": 0.9,
    "beta_2": 0.999,
    "epsilon": 1e-08
}

clf = MLPClassifier(**params) # adam numerical stabilizer


def eval_score(y_true, y_pred):
    return np.sum(confusion_matrix(y_true, y_pred) * cost_matrix)

grid = GridSearchCV(clf, test_params, scoring=make_scorer(eval_score, greater_is_better=False))
grid.fit(X, y)


GridSearchCV(cv=None, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.1, max_iter=75, momentum=0.9,
       nesterovs_momentum=False, power_t=0.0, random_state=None,
       shuffle=True, solver='adam', tol=0, validation_fraction=0.0,
       verbose=False, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'hidden_layer_sizes': [10, 20, 30, 40, 50], 'activation': ['logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'adam']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(eval_score, greater_is_better=False), verbose=0)

Below we have our wrapper class for our MLP classifier. We are using the MLP from scikit-learn because it is both fast and works very well. This wrapper class defines the parameters that we found to be best using the grid search above, and also contains functionality for randomized feature selection. We randomly drop a random number of columns from our array of data so that we are training the different individual classifiers on different datasets. This will improve the performance of the classifiers because they are all looking at permutations of the same data.

In [9]:
class MLPWrapper(object):
    def __init__(self):
        params = {
            "hidden_layer_sizes": 50,
            "activation": 'relu',
            "solver": 'adam',
            "alpha": 1e-4,
            "batch_size": "auto",
            "learning_rate": "constant",
            "learning_rate_init": 0.1,
            "power_t": 0.0,
            "max_iter": 75,
            "shuffle": True,
            "tol":0,
            "verbose": False,
            "warm_start": False,
            "momentum": 0.9,
            "nesterovs_momentum": False,
            "early_stopping": False,
            "validation_fraction": 0.0,
            "beta_1": 0.9,
            "beta_2": 0.999,
            "epsilon": 1e-08
        }
        self.clf = MLPClassifier(**params)
    def fit(self, X, y):
        num_drop = np.random.randint(0, X.shape[1] - 1)
        self.drop_indices = list(set(np.random.randint(0, X.shape[1], num_drop)))
        X_drop = np.delete(X, self.drop_indices, 1)
        self.clf.fit(X_drop, y)
    def predict(self, X):
        X_drop = np.delete(X, self.drop_indices, 1)
        return self.clf.predict(X_drop)
    def predict_proba(self, X):
        X_drop = np.delete(X, self.drop_indices, 1)
        return self.clf.predict_proba(X_drop)
        

Below we have the declaration for the ensemble classifier. It takes the number of classifiers as a parameter and instatiates that many scikit MLP classifiers. We use bagging in the ensemble classifier to sample from the data correctly, and we implement predict_proba to fins the argmax probability between the results from the different classifiers.

In [17]:
class Ensemble(object):
    def __init__(self, num_classifier=25):
        self.num_classifier = num_classifier
        self.classifier_list = []
        
        for x in range(self.num_classifier):
            self.classifier_list.append(MLPWrapper())
            
    def fit(self, X, y):
        for clf in self.classifier_list:
            num_samples = np.random.randint(1, y.shape[0])  #random number of samples to choose from
            random_indices = np.random.randint(1, y.shape[0], num_samples) #get random indices
            clf.fit(X[random_indices], y[random_indices])   #fit based on random indices
    def _predict_yhats(self, X_test):
        yhats = []
        for clf in self.classifier_list:
            yhats.append(clf.predict_proba(X_test))
        return yhats
    def predict(self, X_test):
        yhats = self._predict_yhats(X_test)
        summed = np.sum(yhats, axis=0)
        predictions = np.argmax(summed, axis=1)
        return predictions
    def predict_proba(self, X_test):
        yhats = self._predict_yhats(X_test)
        summed = np.sum(yhats, axis=0) / self.num_classifier
        return summed
            

    

Below we use 10 fold cross validation to randomly generate folds to process with the ensemble classifier. For each fold we run predictions, take the receiver operator characteristic (roc), and the area under the curve (auc).

In [ ]:

ensemble_cost = []
ensemble_confusion = []
roc = []
auc_list = []
for train_index, test_index in cv.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = Ensemble(num_classifier=20)
    clf.fit(X_train, y_train)
    
    yhat = clf.predict(X_test)
    temp_roc = []
    temp_auc = []
    for i in [0, 1, 2, 3]:
        temp = roc_curve(y_test, clf.predict_proba(X_test)[:,i], pos_label=i)
        temp_roc.append(temp)
        temp_auc.append(auc(temp[0], temp[1]))
    auc_list.append(temp_auc)
    roc.append(temp_roc)
    temp_conf = confusion_matrix(y_test, yhat)
    ensemble_cost.append(temp_conf * cost_matrix)
    ensemble_confusion.append(temp_conf)

In [14]:
clf.predict_proba(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3,
       2, 0, 0, 2, 1, 0, 2, 2, 2, 2, 2, 2, 3, 2, 0, 0, 0, 1, 1, 3, 2, 3, 0,
       2, 3, 2, 3, 2, 2, 3, 0, 2, 0, 3, 3, 3, 3, 3, 3, 3, 2, 3, 1, 3, 2, 3,
       0, 3,

Below you can see the boxplot of the performance of the ensemble classifier over the differnet folds. Most of the folds showed great performance! Much greater than the results we got previously running just a single classifier on the data.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.boxplot(ensemble_cost)
plt.show()
print("Medians of Different Folds: ")
for fold in ensemble_cost:
    print(np.median(fold))

Below is a graph of the ROCs for the different classes (low grossing, low-mid, mid-high, high) for each fold of the data (40 in total).

In [ ]:
for x in roc:
    plt.figure()
    plt.plot(x[0], x[1])


Below is the list of AUCs for the various classes and folds

In [ ]:
auc_list

Below is the confusion matrix for our different cross folds of the data. 

In [ ]:
ensemble_confusion

## Exceptional Credit
**NB** Our exceptional credit was randomized feature selection for the individual classifiers. You can find that implemented above.